In [1]:
# blab init
import blab
startup_notebook = blab.blab_startup()
%run $startup_notebook

blab init
blab found local libs at /media/me/DATA/Dropbox/31_Projekte/01_Python/libs
Start Time: 19:15:50


time: 787 ms


# Compare Series and DataFrames
* `compare_series`: Compares the content of two Series.
    Returns several indicators of equality.
* `compare_dataframes`: Compares the content of two DataFrames column by column. Returns several indicators of equality.
* `check_equal`: Compares the content of two DataFrames column by column.
* `compare_col_dtype`: Returns the column names of two DataFrames whose dtype differs
* `get_different_rows`: Returns the rows of two DataFrames that differ

In [2]:
import pandas     as pd 
import bpyth      as bpy
import pandasklar as pak 
pak.Config.set('VERBOSE', False)

time: 1.49 s


## compare_series()

In [3]:
?pak.compare_series

time: 67 ms


Signature: pak.compare_series(s, t, format='dict')
Docstring:
Compares the content of two Series.
Returns a dict or DataFrame 
with info about the Series:
    name:    same name    
    dtype:   same dtype
    len:     same shape        
    nnan:    same number of NaNs   
    content: same content, ignoring index and sort
    sort:    same sort order, ignoring index
    eq:      same relations index->data, ignoring sort
    
File:      /media/me/DATA/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [4]:
# Generate test data
s = pak.random_series( 100, 's')
s = s.apply(pak.decorate, p=0.1) # nan
s

0         NaN
1        iutx
2        yz1m
3         NaN
4     XoLYdÜn
       ...   
95      öj0Wf
96       4u7r
97    TgDUNm2
98    QC7JkÖx
99     sf0M3e
Name: rnd_string, Length: 100, dtype: object

time: 68.6 ms


In [5]:
# Generate compare data
# Play with it!

t = s.copy()
# t.name = 's' # name
# t = t[:99] # len
# t = t.apply(pak.decorate, p=0.5) # nan
#t = t.astype('object') # dtype

t[0], t[1] = t[1], t[0] 
#t = t.sort_values()



time: 72.6 ms


In [6]:
r = pak.compare_series(s,t, format='df')
r

,rnd_string
name,True
dtype,True
len,True
nnan,True
content,True
sort,False
eq,False


time: 80.8 ms


## compare_dataframes()

In [27]:
?pak.compare_dataframes

time: 54.2 ms


Signature: pak.compare_dataframes(df1, df2, reset_index=False, format='df')
Docstring:
Compares the content of two DataFrames column by column.
Returns several indicators of equality:
    name:    True, left_only or right_only. True means the column exists in both DataFrames.     
    dtype:   columns have same dtype     
    nnan:    columns have same number of NaNs   
    content: columns have same content, ignoring index and sort
    sort:    columns have same sort order, ignoring index
    eq:      columns have same relations index->data, ignoring sort    

* reset_index: Set True to ignore index and sort order or the rows.    
* format:      'DataFrame', 'Series', 'dict' or 'bool' (or abbreviations of this):
               Output format. format='DataFrame' will return detailed information.
               format='Series', 'dict' or 'bool' will return a summary.
File:      /media/me/DATA/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [8]:
# Generate test data
s = pak.people()
s

,first_name,age,age_class,postal_code,birthplace,secret,features,history
0,Stephanie,33,30,56664,Bremen,EzUpc5,"{j, Y, s, z}","[A, x]"
1,Niklas,30,30,45612,Berlin,eU6uItzwV,"{T, X}","[A, A, A]"
2,Eileen,26,20,19042,<NA>,OükUSü,"{H, U, Q}","[A, B, C, C]"
3,Ella,37,30,96095,<NA>,0AIaAKawv,"{N, 1}","[b, b, a, b]"
4,David,32,30,<NA>,Bremen,DILwpDSkY,"{d, k, V}","[c, b, a]"
...,...,...,...,...,...,...,...,...
95,Lilli,36,30,57143,Bremen,8Qx8ZsPÖGx,"{M, F}",[]
96,Hans,34,30,68389,Berlin,uÖYBdFkB,"{5, 3, p, D, t}","[A, C, C, B]"
97,Andre,24,20,63104,<NA>,B4n5McSW,"{C, K, L}","[a, b, c]"
98,Heinz,35,30,80438,<NA>,y79xOLg,"{C, h, A}","[A, C, C, B]"


time: 166 ms


In [9]:
# Generate compare data
# Play with it!

t = s.copy()
# t.name = 's' # name
# t = t[:99] # len
# t = t.apply(pak.decorate, p=0.5) # nan
t['age'] = t.age.astype('float') # dtype

#t = t.sort_values()
#t = pak.drop_cols(t, 'age')
t['AAGE'] = 0
#t = t.sort_values(['first_name'])
#t.loc[0,'age'] = None
#t = t.head(50)
t= pak.move_cols(t,'age',-1)
t



,first_name,age_class,postal_code,birthplace,secret,features,history,AAGE,age
0,Stephanie,30,56664,Bremen,EzUpc5,"{j, Y, s, z}","[A, x]",0,33.0
1,Niklas,30,45612,Berlin,eU6uItzwV,"{T, X}","[A, A, A]",0,30.0
2,Eileen,20,19042,<NA>,OükUSü,"{H, U, Q}","[A, B, C, C]",0,26.0
3,Ella,30,96095,<NA>,0AIaAKawv,"{N, 1}","[b, b, a, b]",0,37.0
4,David,30,<NA>,Bremen,DILwpDSkY,"{d, k, V}","[c, b, a]",0,32.0
...,...,...,...,...,...,...,...,...,...
95,Lilli,30,57143,Bremen,8Qx8ZsPÖGx,"{M, F}",[],0,36.0
96,Hans,30,68389,Berlin,uÖYBdFkB,"{5, 3, p, D, t}","[A, C, C, B]",0,34.0
97,Andre,20,63104,<NA>,B4n5McSW,"{C, K, L}","[a, b, c]",0,24.0
98,Heinz,30,80438,<NA>,y79xOLg,"{C, h, A}","[A, C, C, B]",0,35.0


time: 49.8 ms


In [10]:
# Output as DataFrame
pak.compare_dataframes(s,t)

,name,dtype,nnan,content,sort,eq
first_name,True,True,True,True,True,True
age_class,True,True,True,True,True,True
postal_code,True,True,True,True,True,True
birthplace,True,True,True,True,True,True
secret,True,True,True,True,True,True
features,True,True,True,True,True,True
history,True,True,True,True,True,True
age,True,False,True,True,True,True
AAGE,right_only,None,None,False,None,False
(Total),False,False,False,False,False,False


time: 141 ms


In [11]:
# Output as dict
pak.compare_dataframes(s,t, format='dict')

{'name': False,
 'dtype': False,
 'nnan': False,
 'content': False,
 'sort': False,
 'eq': False}

time: 101 ms


In [12]:
# Output as bool
pak.compare_dataframes(s,t, format='bool')

False

time: 82 ms


In [13]:
# This ist the same as check_equal
pak.check_equal(s,t)

False

time: 105 ms


## check_equal()

In [14]:
?pak.check_equal

time: 77.2 ms


Signature: pak.check_equal(obj1, obj2, reset_index=False)
Docstring:
Compare the content of two DataFrames column by column.
Two DataFrames are equal, if 
* they have the same shape
* they have the same column names
* and compare_dataframes(format='bool') is True
File:      /media/me/DATA/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [15]:
df1 = pak.dataframe( [ list('Babykorb'), 
                       list('abfällig'), 
                       list('Abgründe'), 
                       list('Kätzchen'), 
                       list('Landwirt'), 
                       list('lebendig'), 
                       list('Saugrohr'),       
                       list('Trugbild'),                     
                ] )

df2 = pak.dataframe( [ list('Babykorb'), 
                       list('abfällig'), 
                       list('Abgründe'), 
                       list('Kätzchen'), 
                       list('Landwirt'), 
                       list('lebendig'), 
                       list('Saugrohr'),       
                       list('Trugbild'),                     
                ] )

df1

,A,B,C,D,E,F,G,H
0,B,a,b,y,k,o,r,b
1,a,b,f,ä,l,l,i,g
2,A,b,g,r,ü,n,d,e
3,K,ä,t,z,c,h,e,n
4,L,a,n,d,w,i,r,t
5,l,e,b,e,n,d,i,g
6,S,a,u,g,r,o,h,r
7,T,r,u,g,b,i,l,d


time: 195 ms


In [16]:
# Initially the DataFrames are equal
assert pak.check_equal(df1, df2)

time: 162 ms


In [17]:
# One change >> not equal
mask = df2['A'] == 'L'
df2.loc[mask,'A'] = 'R'
assert not pak.check_equal(df1, df2)

time: 96 ms


In [18]:
# Change back >> equal again
mask = df2['A'] == 'R'
df2.loc[mask,'A'] = 'L'
assert pak.check_equal(df1, df2)

time: 58.8 ms


In [19]:
# change column order and row order 
df2 = pak.move_cols(df2,'D').sort_values('D')
df2

,D,A,B,C,E,F,G,H
4,d,L,a,n,w,i,r,t
5,e,l,e,b,n,d,i,g
6,g,S,a,u,r,o,h,r
7,g,T,r,u,b,i,l,d
2,r,A,b,g,ü,n,d,e
0,y,B,a,b,k,o,r,b
3,z,K,ä,t,c,h,e,n
1,ä,a,b,f,l,l,i,g


time: 65.9 ms


In [20]:
# still equal
assert pak.check_equal(df1, df2)

time: 109 ms


## compare_col_dtype()

In [21]:
?pak.compare_col_dtype

time: 51 ms


Signature: pak.compare_col_dtype(df1, df2)
Docstring: Returns the column names of two DataFrames whose dtype differs.
File:      /media/me/DATA/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [22]:
pak.compare_col_dtype(s, t)

['age']

time: 94.3 ms


## get_different_rows()

In [23]:
?pak.get_different_rows

time: 79.8 ms


Signature: pak.get_different_rows(df1, df2, indicator=True)
Docstring:
Returns the rows of two DataFrames that differ. 
Useful for control purposes.
Errors can occur with float.
* additional or missing columns are ignored
File:      /media/me/DATA/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [24]:
# Generate compare data
# Play with it!

t = s.copy()

# change one value
t.loc[0,'age'] = -1

# change order
t = t.sort_values(['first_name'])
t= pak.move_cols(t,'age',-1)

# additional or missing columns are ignored
t['AAGE'] = 0
t = pak.drop_cols(t, 'secret')

t

,first_name,age_class,postal_code,birthplace,features,history,age,AAGE
19,Andre,20,42195,<NA>,"{b, P, v, R}","[b, b, a, b]",25,0
97,Andre,20,63104,<NA>,"{C, K, L}","[a, b, c]",24,0
56,Anja,30,39709,Bremen,"{C, h, A}","[A, B, C, C]",30,0
28,Anna,20,52968,<NA>,"{B, m}",[],28,0
82,Annette,30,74514,Bremen,{c},"[c, b, a]",35,0
...,...,...,...,...,...,...,...,...
93,Vivien,20,97002,Bremen,"{T, X}","[b, b, a, b]",28,0
6,Vivien,20,65252,Bremen,"{T, p, m, V}",[],22,0
51,Volker,20,90432,Berlin,"{I, O}","[A, x]",22,0
72,Wilhelm,30,26820,Berlin,{c},"[A, C, C, B]",32,0


time: 102 ms


In [25]:
pak.get_different_rows(s, t)

,first_name,age,age_class,postal_code,birthplace,secret,features,history,AAGE,_merge
0,Andre,24,20,63104,<NA>,B4n5McSW,"0 {j, Y, s, z}\n1 {T, X}\n...","0 [A, x]\n1 [A, A, A]\n2 ...",NaN,left_only
1,Andre,25,20,42195,<NA>,ILÄzOC,"0 {j, Y, s, z}\n1 {T, X}\n...","0 [A, x]\n1 [A, A, A]\n2 ...",NaN,left_only
2,Anja,30,30,39709,Bremen,H3scÜSZg0u,"0 {j, Y, s, z}\n1 {T, X}\n...","0 [A, x]\n1 [A, A, A]\n2 ...",NaN,left_only
3,Anna,28,20,52968,<NA>,JOÜQÖ,"0 {j, Y, s, z}\n1 {T, X}\n...","0 [A, x]\n1 [A, A, A]\n2 ...",NaN,left_only
4,Annette,32,30,40872,Berlin,QLeütf,"0 {j, Y, s, z}\n1 {T, X}\n...","0 [A, x]\n1 [A, A, A]\n2 ...",NaN,left_only
...,...,...,...,...,...,...,...,...,...,...
195,Vivien,29,20,68389,Bremen,<NA>,"19 {b, P, v, R}\n97 {C, K, L}\n56 ...","19 [b, b, a, b]\n97 [a, b, c]\n56 ...",0.0,right_only
196,Vivien,28,20,97002,Bremen,<NA>,"19 {b, P, v, R}\n97 {C, K, L}\n56 ...","19 [b, b, a, b]\n97 [a, b, c]\n56 ...",0.0,right_only
197,Volker,22,20,90432,Berlin,<NA>,"19 {b, P, v, R}\n97 {C, K, L}\n56 ...","19 [b, b, a, b]\n97 [a, b, c]\n56 ...",0.0,right_only
198,Wilhelm,32,30,26820,Berlin,<NA>,"19 {b, P, v, R}\n97 {C, K, L}\n56 ...","19 [b, b, a, b]\n97 [a, b, c]\n56 ...",0.0,right_only


time: 218 ms


In [26]:
# does not work
#s.compare(t, align_axis=1,keep_shape=False)

time: 24.9 ms
